In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm 
import seaborn as sns

In [100]:
# Read the CSV file
online_retail = pd.read_csv('Online Retail.csv', encoding_errors='ignore')

# Convert the date column to datetime format
online_retail['InvoiceDate'] = pd.to_datetime(online_retail['InvoiceDate'])

# Add the day of the week in the format 'ddd' (e.g., 'Mon', 'Tue', etc.)
online_retail['Day of the Week'] = online_retail['InvoiceDate'].dt.day_name()

#Separted Date and Time
online_retail['InvoiceTime'] = online_retail['InvoiceDate'].dt.time
online_retail['InvoiceDate'] = online_retail['InvoiceDate'].dt.date

# Add Cancel column to identify is invoice has been cancelled
online_retail['Cancelled'] = online_retail['InvoiceNo'][0] == 'C'
online_retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Day of the Week,InvoiceTime,Cancelled
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,Wednesday,08:26:00,False
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,Wednesday,08:26:00,False
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,Wednesday,08:26:00,False
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,Wednesday,08:26:00,False
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,Wednesday,08:26:00,False
...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09,0.85,12680.0,France,Friday,12:50:00,False
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09,2.10,12680.0,France,Friday,12:50:00,False
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09,4.15,12680.0,France,Friday,12:50:00,False
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09,4.15,12680.0,France,Friday,12:50:00,False


In [57]:
#
online_retail['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [58]:
online_retail.dropna(subset=['Country'], inplace=True)
online_retail['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [59]:
#dropping 'Unspecified', 'European Community'
online_retail = online_retail[~online_retail['Country'].isin(('Unspecified', 'European Community'))]
online_retail['Country'].unique()
online_retail


In [101]:
#Drop description and stockcode
online_retail.drop('Description', axis='columns', inplace=True)
#online_retail.add('StockCode', axis='columns', inplace=True)
online_retail


,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Day of the Week,InvoiceTime,Cancelled
0,536365,85123A,6,2010-12-01,2.55,17850.0,United Kingdom,Wednesday,08:26:00,False
1,536365,71053,6,2010-12-01,3.39,17850.0,United Kingdom,Wednesday,08:26:00,False
2,536365,84406B,8,2010-12-01,2.75,17850.0,United Kingdom,Wednesday,08:26:00,False
3,536365,84029G,6,2010-12-01,3.39,17850.0,United Kingdom,Wednesday,08:26:00,False
4,536365,84029E,6,2010-12-01,3.39,17850.0,United Kingdom,Wednesday,08:26:00,False
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,12,2011-12-09,0.85,12680.0,France,Friday,12:50:00,False
541905,581587,22899,6,2011-12-09,2.10,12680.0,France,Friday,12:50:00,False
541906,581587,23254,4,2011-12-09,4.15,12680.0,France,Friday,12:50:00,False
541907,581587,23255,4,2011-12-09,4.15,12680.0,France,Friday,12:50:00,False


In [105]:
online_retail['Total'] = online_retail['UnitPrice'] * online_retail['Quantity']
total = online_retail.groupby('InvoiceNo')['Total'].sum().reset_index()
online_retail
#pd.merge(total, online_retail.drop(['Quantity', 'UnitPrice'], axis='columns').drop_duplicates(), on='InvoiceNo', how='inner')

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Day of the Week,InvoiceTime,Cancelled,Total
0,536365,85123A,6,2010-12-01,2.55,17850.0,United Kingdom,Wednesday,08:26:00,False,15.30
1,536365,71053,6,2010-12-01,3.39,17850.0,United Kingdom,Wednesday,08:26:00,False,20.34
2,536365,84406B,8,2010-12-01,2.75,17850.0,United Kingdom,Wednesday,08:26:00,False,22.00
3,536365,84029G,6,2010-12-01,3.39,17850.0,United Kingdom,Wednesday,08:26:00,False,20.34
4,536365,84029E,6,2010-12-01,3.39,17850.0,United Kingdom,Wednesday,08:26:00,False,20.34
...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,12,2011-12-09,0.85,12680.0,France,Friday,12:50:00,False,10.20
541905,581587,22899,6,2011-12-09,2.10,12680.0,France,Friday,12:50:00,False,12.60
541906,581587,23254,4,2011-12-09,4.15,12680.0,France,Friday,12:50:00,False,16.60
541907,581587,23255,4,2011-12-09,4.15,12680.0,France,Friday,12:50:00,False,16.60


In [116]:
#group
online_retail_group = online_retail.groupby(['InvoiceNo', 'InvoiceDate', 'CustomerID','Country','Day of the Week'], group_keys=True)[['Total']].sum(lambda x: x)

online_retail_group 


,,,,,Total
InvoiceNo,InvoiceDate,CustomerID,Country,Day of the Week,
536365,2010-12-01,17850.0,United Kingdom,Wednesday,139.12
536366,2010-12-01,17850.0,United Kingdom,Wednesday,22.20
536367,2010-12-01,13047.0,United Kingdom,Wednesday,278.73
536368,2010-12-01,13047.0,United Kingdom,Wednesday,70.05
536369,2010-12-01,13047.0,United Kingdom,Wednesday,17.85
...,...,...,...,...,...
C581484,2011-12-09,16446.0,United Kingdom,Friday,-168469.60
C581490,2011-12-09,14397.0,United Kingdom,Friday,-32.53
C581499,2011-12-09,15498.0,United Kingdom,Friday,-224.69
